## data exploration

This notebook has some insights into how we are storing data extracted from programs in the `code-book` project. 

_Goals:_

1. Talk about the `parquet` files we generate via our `code --> tree-sitter --> post-processing` pipeline. 

2. Take a look at the data within the `parquet` files.

3. Brainstorm how we might go about writing another "extractor" that translates `parquet` files into data suitable for Souffle.

4. Brainstorm what analyses we might want to encode using Souffle.

**Note:** the data behind this notebook is data from ~200 Java repositories downloaded from GitHub. When you see examples in this
notebook, they are running against this dataset of Java repositories.

In [1]:
import pyarrow as pa
import pyarrow.dataset as ds
import utils

# First let's "load" our dataset from the `/data/parquet` directory.
# We are storing code, parsed by GitHub's tree-sitter, and augmented with
# some extra data / post-processing of our own. We store this data in the 
# `parquet` format (and, earlier, as compressed `arrow` files). Parquet
# is an on-disk columnar format.
with utils.timing('  + Discovered'):
    print('Discovering data...')
    DATA = ds.dataset('/data/parquet', format='parquet', partitioning='hive')
    
# ^ This step doesn't really load any data into memory (hence: "load"). It just
# discovers all of the files in the `/data/parquet` directory and associates them
# with the DATA dataset object. It also picks up on the partitioning we use (hive).

# Here, let's take a look at some of the files "behind" this dataset.
print()
for file in DATA.files[:5]:
    get_part = lambda n: file.split(n + '=')[1].split('/')[0]
    
    print(file)
    print("Project         : '{}'".format(get_part('project')))
    print("Version         : '{}'".format(get_part('version')))
    print("Language        : '{}'".format(get_part('lang')))
    print("(CST Node) Type : '{}'".format(get_part('type')))
    print()

# In our current setup, the data we glean from programs is partitioned by project / version / language / type 
# the "type" partition corresponds to the types tree-sitter gives each node in the CST it produces. So, if you 
# want to look at all of the `method_invocations` in project=foo, version=1.0.0, lang=java, you'd be looking 
# at a files like `/data/parquet/project=foo/version=1.0.0/lang=java/type=method_invocation/*.parquet`

# Within each of these files is a bunch of data for each of the CST Nodes we got from tree-sitter. Data like 
# the node's corresponding source text, the line/col info, the node's parents in the tree, the node's children, etc.

Discovering data...
  + Discovered: 4.1936s

/data/parquet/project=03bc11ee-12ce-4478-adb3-65862fd8aca0/version=1.0.0/lang=java/type=annotation/5505ffbc84cb41018456fb6728f1057d.parquet
Project         : '03bc11ee-12ce-4478-adb3-65862fd8aca0'
Version         : '1.0.0'
Language        : 'java'
(CST Node) Type : 'annotation'

/data/parquet/project=03bc11ee-12ce-4478-adb3-65862fd8aca0/version=1.0.0/lang=java/type=annotation_argument_list/85ec3f1a777e4eada2d2b97c307c2769.parquet
Project         : '03bc11ee-12ce-4478-adb3-65862fd8aca0'
Version         : '1.0.0'
Language        : 'java'
(CST Node) Type : 'annotation_argument_list'

/data/parquet/project=03bc11ee-12ce-4478-adb3-65862fd8aca0/version=1.0.0/lang=java/type=argument_list/b4f9e84575594b988396dc6bced2405f.parquet
Project         : '03bc11ee-12ce-4478-adb3-65862fd8aca0'
Version         : '1.0.0'
Language        : 'java'
(CST Node) Type : 'argument_list'

/data/parquet/project=03bc11ee-12ce-4478-adb3-65862fd8aca0/version=1.0.0/lang=jav

In [2]:
# Let's actually load some data into memory. Let's load up all of the `method_invocations`. 
method_invocations = None

with utils.timing('  + Loaded `method_invocations`'):
    print('Loading data into memory...')
    method_invocations = DATA.to_table(filter=ds.field('type') == 'method_invocation')

with utils.timing('  + Converted `method_invocations`'):
    print('Converting to Pandas data frame...')
    method_invocations = method_invocations.to_pandas()

Loading data into memory...
  + Loaded `method_invocations`: 5.0106s
Converting to Pandas data frame...
  + Converted `method_invocations`: 25.4425s


In [3]:
# Let's show a nice table of the data we've loaded! 
method_invocations

,def_id,end_idx,file_id,file_path,gid,local_scope_id,name,path,sid,start_idx,text,tid,project,version,lang,type
0,0,1216,4610543992179471878,/tests/src/test/java/com/googlecode/concurrent...,8996080925818024364,-4883349455505471259,expectThat,b'\xac\xf1\x0cR\xf0\x7f\xd8|\x00\x00\x00\x00\x...,915816014246839258,1185,"expectThat("""", actual, matcher)",-8869199984623538133,03bc11ee-12ce-4478-adb3-65862fd8aca0,1.0.0,java,method_invocation
1,0,1339,4610543992179471878,/tests/src/test/java/com/googlecode/concurrent...,-8418875276127241978,6082665075931347454,matches,b'\x06\xf9\x1d\x9e\x8a%*\x8b\x00\x00\x00\x00\x...,-5111272785617050317,1316,matcher.matches(actual),-8685548740712224093,03bc11ee-12ce-4478-adb3-65862fd8aca0,1.0.0,java,method_invocation
2,0,1379,4610543992179471878,/tests/src/test/java/com/googlecode/concurrent...,-2748042253788757095,-6006291969436795529,appendText,b'\x99C(C!\xfe\xdc\xd9\x00\x00\x00\x00\x00\x00...,-5111272785617050317,1349,description.appendText(reason),4442727224634441691,03bc11ee-12ce-4478-adb3-65862fd8aca0,1.0.0,java,method_invocation
3,0,1415,4610543992179471878,/tests/src/test/java/com/googlecode/concurrent...,-2784895481003625046,-6006291969436795529,appendText,b'\xaa50\xa6P\x10Z\xd9\x00\x00\x00\x00\x00\x00...,5750123875547209609,1349,description.appendText(reason)\n .appen...,-407146292422728411,03bc11ee-12ce-4478-adb3-65862fd8aca0,1.0.0,java,method_invocation
4,0,1453,4610543992179471878,/tests/src/test/java/com/googlecode/concurrent...,-453217601328698863,-6006291969436795529,appendDescriptionOf,b'\x11*\xd9*\xfe\xd8\xb5\xf9\x00\x00\x00\x00\x...,6244429602834292489,1349,description.appendText(reason)\n .appen...,-5397845754504347422,03bc11ee-12ce-4478-adb3-65862fd8aca0,1.0.0,java,method_invocation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6788394,0,5159,-4800835878265728631,/tests/src/test/java/com/github/jankroken/comm...,3484866117369331308,5014955498427897893,assertThat,b'l\xfe\x97\'E\xbb\\0\x00\x00\x00\x00\x00\x00\...,-7272094662792547367,5128,assertThat(config.getCommand()),-4369463290157481685,ff868c97-21bf-4b83-b7d2-890da2b28677,1.0.0,java,method_invocation
6788395,0,5196,-4800835878265728631,/tests/src/test/java/com/github/jankroken/comm...,5657557059243847232,5014955498427897893,containsExactly,b'@\xae\xbb\x9c+\xae\x83N\x00\x00\x00\x00\x00\...,-6628874593578140025,5128,assertThat(config.getCommand()).containsExactl...,-6457471156041506837,ff868c97-21bf-4b83-b7d2-890da2b28677,1.0.0,java,method_invocation
6788396,0,5236,-4800835878265728631,/tests/src/test/java/com/github/jankroken/comm...,-9156334927114217520,5014955498427897893,getLogfile,b'\xd0\xd7\xdah\xe5)\xee\x80\x00\x00\x00\x00\x...,2795316785644891405,5217,config.getLogfile(),8310409029240576471,ff868c97-21bf-4b83-b7d2-890da2b28677,1.0.0,java,method_invocation
6788397,0,5237,-4800835878265728631,/tests/src/test/java/com/github/jankroken/comm...,-7582383904141682449,5014955498427897893,assertThat,b'\xef\xd8\xfe*\x03\xf6\xc5\x96\x00\x00\x00\x0...,-7272094662792547367,5206,assertThat(config.getLogfile()),-7118638722662426556,ff868c97-21bf-4b83-b7d2-890da2b28677,1.0.0,java,method_invocation


### The columns within our parquet files

All of our `parquet` files have a consistent encoding: they all have the same columns. Here's a quick breakdown of what columns we have and what they store:

1. `def_id`: the `gid` of the `definition` corresponding to the current node. This is something we add on top of `tree-sitter`. During our post-processing we run a (local, probably bad/imprecise) scope/def-use algorithm. We build a tree of scopes and try to associate nodes that might point to "definitions" with their corresponding definitions. We do this on a per-file basis (so we can only resolve references to things used and defined _in the same file_).

2. `end_idx`: the end of this node in the source file (offset).

3. `file_id`: a unique ID corresponding to the file this node came from.

4. `file_path`: the original file path corresponding to this node.

5. `gid`: **this field is important;** `gid` is our "global identifier" for a given node. Every node should have a unique gid. We use this field as a kind of `key` to refer to nodes in most situations.

6. `local_scope_id`: a unique id corresponding to the scope this node occurs in. Nodes in the same scope should have the same `local_scope_id`.

7. `name`: the "name" of this node, if it has one.

8. `path`: **this field is important;** `path` is a sequence of bytes that stores info on the _parents_ of the given node. I'll talk more about the `path` field later. We use `path` to do efficient queries like: _"find me all calls to foo, that appear as an argument to calls to bar, where bar is nested under an if statement."_

9. `sid`: the "structural identifier" for a given node. Nodes that have the same Concrete Syntax Tree (same tree-sitter sub-tree) should have the same `sid`. So, if you have in a program `2 + (3 * 4)` somewhere and, some other place, `2 + (3 * 4)`, I would expect those two `expressions` to have _different_ `gid`'s but _the same_ `sid`. 

10. `start_idx`: the start of this node in the source file (offset; corresponds to `end_idx`).

11. `text`: the source text associated with this node (all line/source info comes from `tree-sitter`).

12. `tid`: the "textual identifier" for a given node. This is simply a hash of `text`. If two nodes have the _exact same source text_ then they have the same `tid`. They _will not have the same `gid`_.

13. `project`: a unique id for the project this node comes from (extracted from our partition info).

14. `version`: the version for the project this nodes comes from (extracted from our partition info).

15. `lang`: the language for the source file this node comes from (extracted from our partition info).

16. `type`: the type assigned to this node by `tree-sitter` (extracted from our partition info, should be the same for every node loaded this way).

Right now, there is one piece of data I get from `tree-sitter` that is "missing" from this schema: `tree-sitter` (recently) added the ability for people to "name" child nodes. So, for a `method_invocation` in Java, it has both a list of child nodes (`.children`) _and_ some "named" references to child nodes like `.name`, `.object`, and `.arguments`. I can easily add this data back into the schema, but I'm not quite sure how best to encode this. You'll also note I don't have (directly) the `.children` of a given node encoded in this schema. I _can_ do this, but, in general, storing arrays in `parquet` files has been a huge pain. If we want child nodes + names (which I'm guessing we probably will), I'll likely encode this info as bytes and store it in a "packed" byte field (like `path`).

### The path column

There's one column I haven't fully described: it's the `path` column. This column contains bytes that are a "packed" representation of the `parents` of a given node. This column ends up being useful for running accelerated queries against these data frames using a query engine I've been writing. 

(Note: the internal representation of this column is still in flux.)

Path is structured like this:

`([8 byte gid][8 byte def id][8 byte name hash][4 byte start_idx][4 byte end_idx][2 byte node type][2 byte field name][2 byte field index][2 byte padding])*`

Where the above is repeated for _every node starting at the current node and walking "up" the tree until the root node_. The root node does not have the "2 byte field name/index" and, instead, has 6 bytes of padding.

The utility of the path column is that it, in some sense, "denormalizes" common data about a node's parents that can be useful during querying. 


In [4]:
# Let's walk through this data frame again, but this time let's show path-related info
for fileid, path in zip(method_invocations.file_id.values[:5], method_invocations.path.values[:5]):
  
  # Just formatting 
  print()
  indent = ''
  
  decoded = utils.decode_java_path(path)
  # ^ This utility method can "break apart" the path column for us 
  
  for gid, (start_idx, end_idx), info in decoded:
    
    # Here, we handle the "root node" case (remember: root node has 6 bytes padding and no field name/index info)
    if '.' not in info:
      # We've reached the end 
      print('{}Node: {} (gid: {}) [ROOT]'.format(indent, info, gid))
      # We could also get a text_fragement for the root, but let's not bother
      break
    
    # Break this extra info apart for presentation
    node_type, field = info.split('.')
    field, index = field.split('[')
    
    # Clean this up a bit so it reads nices
    index = int(index[:-1])
    field = field[2:]
    
    # Get the corresponding text
    text = utils.get_text_fragement(fileid, start_idx, end_idx)
    
    # Print the data (originally stored in packed path column)
    print('{}Node: {} (gid: {}) is {}{} {} of parent'.format(
      indent, node_type, gid, index, 'th', field 
    ))
    print('{}Text: `{}`'.format(indent, text[:60].replace('\n', '\\n')))
    indent += ' '


Node: method_invocation (gid: 8996080925818024364) is 0th child of parent
Text: `expectThat("", actual, matcher)`
 Node: expression_statement (gid: -2602199411195807079) is 1th child of parent
 Text: `expectThat("", actual, matcher);`
  Node: block (gid: 3269106394605506623) is 0th body of parent
  Text: `{\n    expectThat("", actual, matcher);\n  }`
   Node: method_declaration (gid: 3789734087033307125) is 4th child of parent
   Text: `public <T> void expectThat(T actual, Matcher<? super T> matc`
    Node: class_body (gid: 3205442235092785652) is 0th body of parent
    Text: `{\n  private final Description description;\n  private boolean`
     Node: class_declaration (gid: -3890031431620604298) is 6th child of parent
     Text: `public final class DescriptionBuilder {\n  private final Desc`
      Node: program (gid: 8119154153632592857) [ROOT]

Node: method_invocation (gid: -8418875276127241978) is 0th operand of parent
Text: `matcher.matches(actual)`
 Node: unary_expression (gid: -6

## Thoughts on writing an extractor

We can brainstorm here a bit on what it might take to write an approriate "extractor" for Souffle.

## Thoughts on analyses to write in Souffle

Ideally, I'd like to have the following (in some form):

1. Flow information (data/control flow).

2. Better def/use info (ideally, not scoped to _just within files_).

3. Some form of call-graph info.

I'm not interested in pushing _precision_ as much as I am interested in pushing _avaliability_. That is, I'd rather have _some info_ than "percise" info. My (limited) grad-student experience with program analysis has been that most of the papers I read have techniques that (rarely) make it into a real tool. I'd rather have fuzzy/impercise info that we can surface through our querying API than limited/costly (but more precise) info.

Also of interest: I have "ML integrations" I'm working on separately. Things like embeddings for nodes/code fragements. If those can be useful _within analyses_ we could explore such an intergration. Or, other way around, if there are ways to build anaylses that help boulster the "learning" we can do, I'd be interested in that. (And, regardless, this is closer to my expertise: I've spent most of my Ph.D. doing, roughly, "learning from code" or "learning from structured data" --- sometimes it's "real" ML, but mostly it's been embeddings / more traditional "mining" techniques.) 